In [ ]:
# dependencies import
from common_dependencies import *
logger = logging.getLogger('main.model_test')

# Загрузка модели для тестирования

In [ ]:
bins = 11
step = 1/(bins-1)
for value in np.arange(0,1+step/2,step):
    print(value)

In [ ]:
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)

## Поиск модели по идентификатору

In [ ]:
MODEL_VER = 14
MODEL_NUM = 5
EPOCH = 29
# 21 - best Recall
# 29 - best PR_AUC
# 30 - best loss
# 31
#CHECK_NUMS = [21]

model_pathes = [path for path in pathlib.Path(f'networks/CNN').rglob(f'*.keras') 
                     if re.search(rf'id=v0*{MODEL_VER}n0*{MODEL_NUM}.*epoch={EPOCH}',path.name)]

if len(model_pathes) == 1:
    PATH_TO_MODEL = model_pathes[0]
else:
    print(model_pathes)
    raise ValueError('Few or none model have been found instead of one')

print(f'{PATH_TO_MODEL=}')

## Загрузка модели

In [ ]:
tholds = np.round(np.arange(0.01,1,0.01),decimals=2).tolist()
print(len(tholds))
print(tholds)

In [ ]:
model = keras.models.load_model(PATH_TO_MODEL)

CMP_learning_rate = 0.005 # шаг сходимости back propogation
#CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
CMP_solver = keras.optimizers.SGD(CMP_learning_rate) # оптимизатор
CMP_loss_funcs = keras.losses.BinaryCrossentropy() #BinaryCrossentropy() 
CMP_metrics = [keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives'),     
               keras.metrics.Precision(name='Precision'),#, thresholds=tholds),
               keras.metrics.Recall(name='Recall'),#, thresholds=tholds),
               keras.metrics.AUC(name='ROC_AUC', curve='ROC'),
               keras.metrics.AUC(name='PR_AUC', curve='PR')]

model.compile(optimizer=CMP_solver, loss=CMP_loss_funcs, metrics=CMP_metrics)

'''print(model.summary())

tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)''';

# Загрузка данных для тестирования

## Параметры  выборок

In [ ]:
XSHIFT = 200
MAX_VAL = 1000
CROP_SIZE = model.inputs[0].shape[1]
dataset_desc =  {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(100,0,270,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT)
                         ,),
                 'val': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,40),SlidingCrop(CROP_SIZE,1),XSHIFT)
                        ,),
                 'test': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,100,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                          DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(370,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                          DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,40,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT)
                            ,)}

## Загрузка данных

In [ ]:
#dataset = {'val':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['val'])]))}

## Размерности данных в обучающих выборках

In [ ]:
'''logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')''';

## Какие части 2 файлов с данными относятся к конкретным выборкам (к тренировочной, тестовой, валидационной)

In [ ]:
#выборка данных
# show parts took for learning
'''all_rects = {'run_1': {'train':None,'val':None,'test':None},  #
             'run_2': {'train':None,'val':None,'test':None}} #
rects_colors = {'train':'red', 'val':'green', 'test':'yellow'}

for run_name in all_rects.keys():
    x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[run_name])
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.4, edgecolor='white')
                                                  for dataset_part in dataset_part_desc]
        
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')
    else:
        dw.draw_defects_map(y_df, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')''';

# Тестирование модели

In [ ]:
dataset = {'train':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['train'])]))}
train_res = model.evaluate(dataset['train']['x'], 
                     dataset['train']['bin'], 
                     batch_size=32, 
                     verbose=1, return_dict=True)

# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

In [ ]:
dataset = {'val':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['val'])]))}
val_res = model.evaluate(dataset['val']['x'], 
                     dataset['val']['bin'], 
                     batch_size=32, 
                     verbose=1, return_dict=True)

# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

In [ ]:
dataset = {'test':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['test'])]))}
test_res = model.evaluate(dataset['test']['x'], 
                     dataset['test']['bin'], 
                     batch_size=32, 
                     verbose=1, return_dict=True)

# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

In [ ]:
'''print('|'*8,'train','|'*8)#,'\n',train_res,'\n')
print(*[(key,value) for (key,value) in train_res.items()],sep='\n')
print()
print('|'*8,'val','|'*8)#,'\n',train_res,'\n')
print(*[(key,value) for (key,value) in val_res.items()],sep='\n')
print()
print('|'*8,'test','|'*8)#,'\n',train_res,'\n')
print(*[(key,value) for (key,value) in test_res.items()],sep='\n')
print()''';

## Запись результатов в датафрейм

In [ ]:
df = pd.DataFrame(data=np.stack([train_res['Precision'].numpy(),
                                       train_res['Recall'].numpy(),
                                       val_res['Precision'].numpy(),
                                       val_res['Recall'].numpy(),
                                       test_res['Precision'].numpy(),
                                       test_res['Recall'].numpy()],axis=1),
                        columns=pd.MultiIndex.from_product([['train','val','test'],['precision','recall']], names=['dataset','metric']), 
                        index=pd.Index(tholds,name='treshold'))

## Общие параметры датафрейма 

In [ ]:
display(df)

In [ ]:
params_df = pd.concat([df.max(),df.min(),df.mean()],axis=1,keys=['max','min','mean'])
display(params_df)

In [ ]:
df_draw = df.swaplevel(axis=1)
display(df_draw)
FONT_SIZE = 15

fig, axes = plt.subplots(2,1)

fig.set_figwidth(12)
fig.set_figheight(8)

df.swaplevel(axis=1)['precision'].plot(kind='line',
                                       ax=axes[0])
df.swaplevel(axis=1)['recall'].plot(kind='line',
                                    ax=axes[1])

axes[0].set_ylabel('Precision',fontsize=FONT_SIZE)
axes[0].set_title('Precision',fontsize=FONT_SIZE)

axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)
axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)

for ax in axes:
    ax.grid(True)
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    ax.margins(0.05)
    ax.legend(fontsize = FONT_SIZE)
    ax.set_xlabel('Threshold',fontsize=FONT_SIZE)
    ax.grid(True)
        
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.3)
plt.show()

'''plt.plot(history.history[key], 
         label='Train dataset',  linewidth=1.5, color='blue')
if key != 'learning_rate':
    plt.plot(history.history[f'val_{key}'], linestyle = '--', 
         label='Validation dataset',  linewidth=3, color='red')

ax.set_xlabel('Epoch number', fontsize=FONT_SIZE)
ax.set_ylabel(f'{key} value', fontsize=FONT_SIZE)
ax.set_title(f"Learning process {key} plot", fontsize=FONT_SIZE, pad=15)

ax.patch.set_alpha(0)

#  Устанавливаем форматирование делений:
ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)

# Вывод и настройка сетки
ax.minorticks_on()
ax.grid(which='major', linewidth=2)
ax.grid(which='minor', color = 'gray', linestyle = ':')

ax.legend(fontsize = FONT_SIZE, facecolor = "white")''';

### Средние параметры по всем выборкам и метрикам

In [ ]:
mean_df = df.copy()
mean_df = mean_df.stack(0, future_stack=True)
mean_df['presicion_plus_recall'] = mean_df['precision'] + mean_df['recall']
mean_df = mean_df.query('precision > 0.6 & recall > 0.7') #для модели 30 эпохи
#mean_df = mean_df.query('precision > 0.576 & recall > 0.64') #для модели 29 эпохи
mean_df = mean_df.unstack('dataset')
mean_df = mean_df.dropna()
print('Потенциальные результаты')
display(mean_df[mean_df['presicion_plus_recall']['train']>=1.6]) #для модели 30 эпохи
#display(mean_df[mean_df['presicion_plus_recall']['train']>=1.6332]) #для модели 29 эпохи

### Лучший precision при не позорном recall

In [ ]:
pres_df = df.copy()
pres_df = pres_df.stack(0, future_stack=True)
pres_df['presicion_plus_recall'] = pres_df['precision'] + pres_df['recall']
pres_df = pres_df.query('precision >= 0.647 & recall > 0.2') #для модели 30 эпохи
#pres_df = pres_df.query('precision > 0.5777 & recall > 0.2') #для модели 29 эпохи
pres_df = pres_df.unstack('dataset')
pres_df = pres_df.dropna()
print('Потенциальные результаты')
display(pres_df) #для модели 30 эпохи
#display(pres_df[pres_df['precision']['train']>=0.792]) #для модели 29 эпохи

### Лучший recall при не позорном precision

In [ ]:
recall_df = df.copy()
recall_df = recall_df.stack(0, future_stack=True)
recall_df['presicion_plus_recall'] = recall_df['precision'] + recall_df['recall']
recall_df = recall_df.query('precision > 0.2 & recall > 0.8') #для модели 30 эпохи
#recall_df = recall_df.query('precision > 0.2 & recall > 0.82') #для модели 29 эпохи
recall_df = recall_df.unstack('dataset')
recall_df = recall_df.dropna()
print('Потенциальные результаты')
display(recall_df)

## Сравнение фаворитов

In [ ]:
comp_df = df.copy()
comp_df = comp_df.loc[[0.01,0.09,0.26]] #для модели 30 эпохи
#comp_df = comp_df.loc[[0.01,0.28,0.4]] #для модели 29 эпохи
comp_df = comp_df.stack(0, future_stack=True)
comp_df = comp_df.unstack('treshold')
display(comp_df)
comp_df = comp_df.stack(1, future_stack=True)
comp_df = comp_df.unstack('dataset')
comp_df['best_in'] = ['best_recall','best_mean_vals','best_precision']
comp_df.set_index('best_in', append=True, inplace=True)
display(comp_df)

In [ ]:
#display(comp_df)
#comp_df.to_excel(f'model_id=v{MODEL_VER}n{MODEL_NUM}_epoch={EPOCH}_best_thresholds.xlsx')

In [ ]:
'''df_29 = pd.read_excel(f'model_id=v14n05_epoch=29_best_thresholds.xlsx', index_col=[0,1], header=[0,1])
df_30 = pd.read_excel(f'model_id=v14n05_epoch=30_best_thresholds.xlsx', index_col=[0,1], header=[0,1])
#best_df = pd.concat([df_29,df_30], axis=1, keys=[29,30], names=['epoch'])
#best_df = best_df.droplevel('treshold')
display(df_29)
display(df_30)
#display(best_df)
#display(best_df.stack(1,future_stack=True).reorder_levels(["best_in", "treshold","metric"]).stack(0, future_stack=True))
#display(best_df.stack(0, future_stack=True).stack(1, future_stack=True).unstack('treshold').reorder_levels(["best_in", "epoch","dataset",]))'''

In [ ]:
'''FONT_SIZE = 15

fig, axes = plt.subplots(2,1)

fig.set_figwidth(12)
fig.set_figheight(18)

#df_29['precision'].hist(ax=axes[0], bins=3)
#df_29['recall'].hist(ax=axes[0], bins=3)

df_29.droplevel('treshold')['precision'].plot(kind='line', 
                        ax=axes[0], marker='o', ms=8, mec='blue', mfc='blue')
df_30.droplevel('treshold')['precision'].plot(kind='line', 
                        ax=axes[0], marker='o', ms=8, mec='red', mfc='red')
df_29.droplevel('treshold')['recall'].plot(kind='line', 
                     ax=axes[1], marker='o', ms=8, mec='blue', mfc='blue')
df_30.droplevel('treshold')['recall'].plot(kind='line', 
                     ax=axes[1], marker='o', ms=8, mec='red', mfc='red')

axes[0].set_ylabel('Precision (blue dots - 29 epoch model, red - 30 epoch model)',fontsize=FONT_SIZE)
axes[0].set_title('Precision (blue dots - 29 epoch model, red - 30 epoch model)',fontsize=FONT_SIZE)

axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)
axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)

for ax in axes:
    ax.grid(True)
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    ax.margins(0.05)
    ax.legend(fontsize = FONT_SIZE)
    ax.set_xlabel('Best in',fontsize=FONT_SIZE)
    ax.grid(True)
        
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.3)
plt.show()'''

In [ ]:
'''FONT_SIZE = 15

fig, axes = plt.subplots(2,1)

fig.set_figwidth(12)
fig.set_figheight(8)

comp_df.stack(1, future_stack=True).unstack('dataset')['precision'].plot(kind='line', ax=axes[0], marker='o', ms=8)
comp_df.stack(1, future_stack=True).unstack('dataset')['recall'].plot(kind='line', ax=axes[1], marker='o', ms=8)

axes[0].set_ylabel('Precision',fontsize=FONT_SIZE)
axes[0].set_title('Precision',fontsize=FONT_SIZE)

axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)
axes[1].set_ylabel('Recall',fontsize=FONT_SIZE)

for ax in axes:
    ax.grid(True)
    ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    ax.margins(0.05)
    ax.legend(fontsize = FONT_SIZE)
    ax.set_xlabel('Threshold',fontsize=FONT_SIZE)
    ax.grid(True)
        
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.3)
plt.show()''';

In [ ]:
'''temp_df = temp_df.stack(1, future_stack=True)
display(temp_df)

temp_df = temp_df.swaplevel(axis=1)
display(temp_df)''';

In [ ]:
'''FONT_SIZE=15

fig, axes = plt.subplots()
fig.set_figwidth(22)
fig.set_figheight(8)

axes.set_title('Precision-Recall curve',fontsize = FONT_SIZE)

axes.plot(tholds, res['Precision'], label='Precision')
axes.plot(tholds, res['Recall'], label='Recall')

axes.set_xlabel('Trashold',fontsize = FONT_SIZE)
axes.set_ylabel('Recall and Recall',fontsize = FONT_SIZE)

axes.grid(True)
axes.tick_params(axis='both', which='both', labelsize = FONT_SIZE)
axes.margins(0.05)
axes.legend(fontsize = FONT_SIZE)
axes.legend(fontsize = FONT_SIZE, facecolor = "white")

plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.15, hspace=0.1)
plt.show()''';

In [ ]:
#dataset = {'test':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['test'])] ))}

In [ ]:
'''metric_dict_14 = {'train':None,'val':None,'test':None}
for dataset_part_name, dataset_part in dataset.items():
    print('|'*8+dataset_part_name+'|'*8)
    metrics = [keras.metrics.AUC(name='PR_AUC', curve='PR'),
               keras.metrics.AUC(name='ROC_AUC', curve='ROC'),
               keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
               keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives')]

    for (x_buffer,  y_buffer) in zip(*dataset_part.values()):
        x_buffer = np.pad(x_buffer, ((0,0),(1,1),(1,1),(0,0)), 'reflect')
        #print(x_buffer.shape)
        res = load_model.predict(x_buffer, verbose=0)
        for metric in metrics:
            metric.update_state(y_buffer, res)

    metric_dict_14[dataset_part_name] = {metric.get_config()["name"]:metric.result() for metric in metrics}
    for metric in metrics:
        #print(f'{metric.get_config()["name"]}:{metric.result():.6f}')
        metric.reset_state()''';

In [ ]:
'''CROP_SIZE = 16

dataset_desc =  {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),),
                 'val': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,40),SlidingCrop(CROP_SIZE,1),XSHIFT),),
                 'test': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,40,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),)}

dataset = {'train':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['train'])] )),
           'val':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['val'])] )),
           'test':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['test'])] ))}''';

In [ ]:
'''metric_dict_16 = {'train':None,'val':None,'test':None}
for dataset_part_name, dataset_part in dataset.items():
    print('|'*8+dataset_part_name+'|'*8)
    metrics = [keras.metrics.AUC(name='PR_AUC', curve='PR'),
               keras.metrics.AUC(name='ROC_AUC', curve='ROC'),
               keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
               keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives')]

    for (x_buffer,  y_buffer) in zip(*dataset_part.values()):
        #x_buffer = np.pad(x_buffer, ((0,0),(1,1),(1,1),(0,0)), 'reflect')
        #print(x_buffer.shape)
        res = load_model.predict(x_buffer, verbose=0)
        for metric in metrics:
            metric.update_state(y_buffer, res)

    metric_dict_16[dataset_part_name] = {metric.get_config()["name"]:metric.result() for metric in metrics}
    for metric in metrics:
        #print(f'{metric.get_config()["name"]}:{metric.result():.6f}')
        metric.reset_state()''';

In [ ]:
'''for (data_part1, data_part2) in zip(metric_dict_14.keys(), metric_dict_16.keys()):
    print('|'*8+data_part1+'|'*8)
    for ((metric_name1, metric_res1), (metric_name2, metric_res2)) in zip(metric_dict_14[data_part1].items(), metric_dict_16[data_part2].items()):
        print('|'*8+metric_name1+'|'*8)
        print(f'Вход 16 на 16: {metric_res2}')
        print(f'Вход 14 на 14: {metric_res1}')
        print(f'Метрика для 14 на 14 это {metric_res1/metric_res2*100:.2f}% от метрики 16 на 16')
        print()''';

In [ ]:
'''# тест модели
res_list = list()
for crop_step in CROP_STEPS:
    (test_x_time_gen, 
     test_x_amp_gen, 
     test_y_binary_gen,
     test_y_depth_gen) = get_dataset_gen(PATH_TO_DATA[f'run_{run}'], XSHIFT, DataCrop(None,None,None,200), CROP_SIZE, 20)
    
    test_x_time = np.stack([crop for crop in test_x_time_gen])
    test_x_amp = np.stack([crop for crop in test_x_amp_gen])
    test_y_binary = np.array([binary for binary in test_y_binary_gen])
    test_y_depth = np.array([depth for depth in test_y_depth_gen])
    
    res = load_model.evaluate([test_x_time, test_x_amp], test_y_binary, batch_size = 32, return_dict=True)
    res_list.append({'res':res,'crop_size':CROP_SIZE,'crop_step':crop_step})''';

In [ ]:
'''for line in res_list:
    print(f'crop_size: {line["crop_size"]}')
    print(f'crop_size: {line["crop_step"]}')
    print(line['res'])
    print()''';